In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_sale_price_store_attr
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
 
 AS
SELECT 
  LTRIM('0', customer) store_id,
  sales_organization sale_org,
  distribution_channel,
  division,
  price_group, 
  price_list
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmdm01
WHERE proc_date = (SELECT MAX(proc_date) FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdmdm01)
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer, sales_organization, distribution_channel, division ORDER BY proc_date DESC) = 1
""")